# odc-stac tutorial

In this tutorial we will use Python libraries to find and load land cover data from the freely available [Impact Observatory Annual Land Use Land Cover](https://planetarycomputer.microsoft.com/dataset/io-lulc-annual-v02) product.
After loading the data, we will export it as a cloud-optimised GeoTiff.
This will allow you to further view or work with the data in GIS software and other tools.

During the tutorial, we will:

* Specify our search in terms of:

  * what (data provider and satellite)
  * where (area of interest)
  * when (date range)
* Use `pystac-client` to connect to a Spatio-Temporal Asset Catalog (STAC) 
  endpoint and search for data matching our what, where, and when
* Use `odc-stac` to load the matching data into memory
* Visualise and export the data

## Requirements
Please keep a copy of the [tutorial instructions](https://opendatacube.readthedocs.io/en/latest/tutorials/odc-stac.html) open.

Follow the instructions for each labelled section. 
When you have finished entering code into a cell, press `Shift+Enter` on your keyboard to run the cell.

## Python imports

In [ ]:
import geopandas as gpd
from odc.stac import load
import planetary_computer
from pystac_client import Client

import warnings
warnings.filterwarnings('ignore')


## Set up query parameters

### Area of interest

In [ ]:
aoi = gpd.read_file("aoi.geojson")

aoi_geometry = aoi.iloc[0].geometry

### Date range

In [ ]:
start_date = "2017-01-01"
end_date = "2023-01-01"

date_query = (start_date, end_date)

### Catalogs, collections, and items

In [ ]:
catalog_query = "https://planetarycomputer.microsoft.com/api/stac/v1/"
collections_query = ["io-lulc-annual-v02"]
assets_query = ["data"]

## Connect to catalog and find items

This step returns STAC items that match the query, each of which points to a Cloud Optimised GeoTIFF (COG) that contains the relevant data

In [ ]:
# Connect to the catalog
stac_client = Client.open(
    catalog_query, 
    modifier=planetary_computer.sign_inplace,
)

### Search for items

`pystac-client` is used to search for items that match the query. The search function has a few arguments:

- `collections`: a list of collections to search
- `intersects`: an area of interest to search. Any item intersecting the geometry will be returned.
- `datetime`: a date range to search

In [ ]:
# Search for items matching the query
items = stac_client.search(
    collections=collections_query,
    intersects=aoi_geometry,
    datetime=date_query,
).item_collection()

print(f"Found {len(items)} items")

Once returned, it is possible to view relevant metadata for all discovered items.

In [ ]:
items

## Load items with odc-stac

Once the location of the items is known, `odc-stac` can load these items. The load function has a few arguments:

- `items`: a list of STAC items to load
- `bands`: a list of assets to load for each item
- `geopolygon`: the area of interest to load the data for
- `crs`: the coordinate reference system to use. `utm` will provide the most appropriate UTM projection
- `resolution`: the resolution to load the items at, in the same units as the `crs`. If using `utm` the unit is metres
- `groupby`: using `solar_day` here will merge all items captured on the same solar day into one image


In [ ]:
ds_filtered = load(
    items,
    bands=assets_query,
    geopolygon=aoi_geometry,
    crs="utm",
    resolution=30
)

ds_filtered

## Visualise loaded data

If an `xarray` has been loaded using the Open Data Cube ecosystem, it comes with an `xarray` extension `xarray.odc` that allows you to access additional geospatial metadata and functions.

Learn more at [odc-geo](https://odc-geo.readthedocs.io/en/latest/intro-xr.html).

In [ ]:
# Get a single image at the first time step, and remove the time dimension
ds_filtered.data.plot.imshow(col="time", col_wrap=3)

In [ ]:
# Create an RGB image using the .odc. extension
visualisation = image.odc.to_rgba()

# Display the RGB image on a map
visualisation.odc.explore()

## Export loaded 

The `.odc` extension is also useful for saving out data as Cloud Optimised GeoTIFFs (COGs).

In [ ]:
visualisation.odc.write_cog(
    "sentinel2_example.tif",
    overwrite=True
)

## Tutorial Complete!

Try loading your saved COG in QGIS or another GIS program.